<a href="https://colab.research.google.com/github/dp913/CSE-541-Computer-Vision-2023-Group-8/blob/master/Code/Gastrointestinal_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries

import tensorflow
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model, load_model
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import cv2
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Image Classification

In [ ]:
# Define image size and batch size
img_height, img_width = (224,224)
batch_size = 32

# Load the train, test and validation sets
train_data_dir = "/content/drive/MyDrive/split_dataset/train"
val_data_dir = "/content/drive/MyDrive/split_dataset/val"
test_data_dir = "/content/drive/MyDrive/split_dataset/test"


In [ ]:
# Image Augmentation using the ImageDataGenerator 

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range=0.3,       # specifies the range for randomly applying shear transformations to the input images during data augmentation
                                   zoom_range=0.2,        # specifies the range for randomly applying zoom transformations to the input images during data augmentation.
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,  # specifies whether to randomly flip the images horizontally during data augmentation
                                   validation_split=0.4)  # specifies the proportion of the training data that should be reserved for validation during model training.

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),  # The dimensions to which all images found will be resized.
    batch_size = batch_size,              # Size of the batches of data (default: 32).
    class_mode = 'categorical',           # Determines the type of label arrays that are returned: "categorical" will be 2D one-hot encoded labels
    subset = 'training')                  # Subset of data ("training" or "validation") if validation_split is set in ImageDataGenerator.

valid_generator = train_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'validation')

test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size = 1,
    class_mode = 'categorical',
    subset = 'validation')


Found 949 images belonging to 4 classes.
Found 203 images belonging to 4 classes.
Found 211 images belonging to 4 classes.


In [ ]:
# Load the pre-trained ResNet50 model from keras
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output

# Apply Global Average Pooling on the output layer
x = GlobalAveragePooling2D()(x)

# Add a layer with 1024 neurons and relu activation
x = Dense(1024, activation='relu')(x)

# Define the output layer with 4 classes and Softmax activation
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Add the output layer to the model
resnet_model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
  layer.trainable = False

# Compile the Model with appropriate optimizer and loss function
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

# Train the model on train set
resnet_model.fit(train_generator, epochs=30, validation_data = valid_generator)

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/30
30/30 [==============================] - 426s 14s/step - loss: 1.6097 - accuracy: 0.7713 - val_loss: 0.5724 - val_accuracy: 0.8867
Epoch 2/30
30/30 [==============================] - 31s 1s/step - loss: 0.3246 - accuracy: 0.9094 - val_loss: 0.2229 - val_accuracy: 0.9163
Epoch 3/30
30/30 [==============================] - 30s 1s/step - loss: 0.1846 - accuracy: 0.9283 - val_loss: 0.3817 - val_accuracy: 0.8719
Epoch 4/30
30/30 [==============================] - 30s 1s/step - loss: 0.1509 - accuracy: 0.9473 - val_loss: 0.2712 - val_accuracy: 0.9212
Epoch 5/30
30/30 [==============================] - 32s 1s/step - loss: 0.1247 - accuracy: 0.9568 - val_loss: 0.2708 - val_accuracy: 0.9310
Epoch 6/30
30/30 [==============================] - 36s 1s/step - loss: 0.1589 - accuracy: 0.9484 - val_loss: 0.2434 - val_accuracy: 0.9163
Epoch 7/30
30/30 [==============================] - 35s 1s/step - loss: 0.1247 - accuracy: 0.

In [ ]:
# Evaluate the model on test set

test_loss, test_acc = resnet_model.evaluate(test_generator)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

211/211 [==============================] - 117s 553ms/step - loss: 0.2759 - accuracy: 0.9242
Test loss: 0.27587711811065674
Test accuracy: 0.9241706132888794


In [ ]:
# Save the model
resnet_model.save('/content/drive/My Drive/models/resnet50_images.h5')

#Video Classification

In [ ]:
# Video dataset path
dataset_path = '/content/drive/MyDrive/Video_Dataset3'

In [ ]:
# Load model
model = load_model('/content/drive/MyDrive/models/resnet50_images.h5')

flatten_layer = tensorflow.keras.layers.Flatten()(model.output)
output_layer = tensorflow.keras.layers.Dense(1, activation='relu')(flatten_layer)

# Create the full model by combining the ResNet50 model with the classification layers
full_model = tensorflow.keras.models.Model(inputs=model.input, outputs=output_layer)

In [ ]:
# Define a function to extract frames from videos and preprocess them for use with the ResNet50 model
def preprocess_video(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    cap.release()
    print("Frames = ", len(frames))
    frames = np.array(frames) / 255.0
    return frames

In [ ]:
# Iterate over the videos in the dataset and extract their features using the ResNet50 model
features = []
labels = []
for label in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label)
    print("Label: ",label)
    for video_file in os.listdir(label_path):
        video_path = os.path.join(label_path, video_file)
        frames = preprocess_video(video_path)
        video_features = full_model.predict(frames)
        video_features = np.max(video_features, axis=0)
        features.append(video_features)
        labels.append(label)

Label:  Polyps
Frames =  199
7/7 [==============================] - 1s 98ms/step
Frames =  567
18/18 [==============================] - 2s 96ms/step
Frames =  143
5/5 [==============================] - 0s 90ms/step
Frames =  449
15/15 [==============================] - 1s 98ms/step
Frames =  752
24/24 [==============================] - 2s 98ms/step
Frames =  387
13/13 [==============================] - 1s 97ms/step
Frames =  295
10/10 [==============================] - 1s 91ms/step
Frames =  601
19/19 [==============================] - 2s 93ms/step
Frames =  677
22/22 [==============================] - 2s 91ms/step
Frames =  370
12/12 [==============================] - 1s 91ms/step
Label:  healthy
Frames =  300
10/10 [==============================] - 1s 91ms/step
Frames =  300
10/10 [==============================] - 1s 86ms/step
Frames =  300
10/10 [==============================] - 1s 92ms/step
Frames =  300
10/10 [==============================] - 1s 91ms/step
Frames =  300
10/10 [

In [ ]:
# Convert the features and labels to numpy arrays and one-hot encode the labels

features = np.array(features)
# print("labels:",labels)
new_labels = labels
#labels = np.array(labels)
new_labels = np.array(new_labels)
for i in range(len(new_labels)):
  if new_labels[i] == 'healthy':
    new_labels[i]=0
  elif new_labels[i]=='Polyps':
    new_labels[i]=1
    
# print("new_labels:",new_labels)

new_labels = np.eye(len(np.unique(new_labels)))[new_labels.astype(int)]

labels: ['Polyps', 'Polyps', 'Polyps', 'Polyps', 'Polyps', 'Polyps', 'Polyps', 'Polyps', 'Polyps', 'Polyps', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy']
new_labels: ['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']


In [ ]:
print(new_labels)
# final_labels = []
for i in range(len(new_labels)):
  final_labels.append(int(new_labels[i][1]))
# print(final_labels)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# Split the data into training and testing sets

X_train, X_test, Y_train, Y_test = train_test_split(features, final_labels, test_size=0.2, random_state=2)

In [ ]:
# Train the MLP classifier model on the training set

clf = MLPClassifier(activation='relu', alpha=0.0001, solver='adam', hidden_layer_sizes=(64,), max_iter=1000)
clf.fit(X_train.reshape((X_train.shape[0], -1)), Y_train)


MLPClassifier(hidden_layer_sizes=(64,), max_iter=1000)

In [ ]:
# Evaluate the model
score = clf.score(X_train.reshape((X_train.shape[0], -1)), Y_train)
print('Train accuracy:', score)
score = clf.score(X_test.reshape((X_test.shape[0], -1)), Y_test)
print('Test accuracy:', score)

Train accuracy: 0.6818181818181818
Test accuracy: 0.5


###Hyperparameter Tuning of the MLP classifier

In [ ]:
# define a grid of hyperparameters to search over
param_grid = {
    'hidden_layer_sizes': [(128,), (256,), (512,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'sgd', 'lbfgs'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': [500, 1000, 2000]
}

# perform a grid search to find the best hyperparameters
from sklearn.model_selection import GridSearchCV
clf = MLPClassifier()
grid = GridSearchCV(clf, param_grid, cv=3, verbose=2, n_jobs=-1)
grid.fit(X_train.reshape((X_train.shape[0], -1)), Y_train)

# print the best hyperparameters and validation score
print('Best hyperparameters:', grid.best_params_)
print('Validation accuracy:', grid.best_score_)

Fitting 3 folds for each of 729 candidates, totalling 2187 fits
Best hyperparameters: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (128,), 'learning_rate': 'constant', 'max_iter': 500, 'solver': 'adam'}
Validation accuracy: 0.6845238095238096


In [ ]:
# Train the MLP classifier model on the training set with the best hyperparameters
clf2 = MLPClassifier(activation= 'relu',
                    alpha= 0.0001,
                    hidden_layer_sizes= (128,),
                    learning_rate= 'constant',
                    max_iter= 500,
                    solver= 'adam')
clf2.fit(X_train.reshape((X_train.shape[0], -1)), Y_train)

MLPClassifier(hidden_layer_sizes=(128,), max_iter=500)

In [ ]:
# Evaluate the model
score = clf2.score(X_train.reshape((X_train.shape[0], -1)), Y_train)
print('Train accuracy:', score)
score = clf2.score(X_test.reshape((X_test.shape[0], -1)), Y_test)
print('Test accuracy:', score)

Train accuracy: 0.6818181818181818
Test accuracy: 0.5
